In [1]:
!pip install -U yfinance
!pip install quandl

!pip install -U statsmodels
!pip install --upgrade scipy

!pip install fracdiff
!pip install stumpy

!pip install optuna
!pip install flaml
!pip install tpot

!pip install tpot
!pip install featexp

!pip install sktime
!pip install pytrends

  Using cached statsmodels-0.13.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.8 MB)
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.1
    Uninstalling statsmodels-0.12.1:
      Successfully uninstalled statsmodels-0.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.9.0 requires statsmodels<=0.12.1, but you have statsmodels 0.13.1 which is incompatible.
fracdiff 0.7.0 requires numpy<2.0.0,>=1.21.3, but you have numpy 1.19.3 which is incompatible.
  Using cached numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.3
    Uninstalling numpy-1.19.3:
      Successfully uninstalled numpy-1.19.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Th

     |████████████████████████████████| 15.3 MB 8.9 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.4
    Uninstalling numpy-1.21.4:
      Successfully uninstalled numpy-1.21.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.20.3 which is incompatible.
sktime 0.9.0 requires numpy<=1.19.3, but you have numpy 1.20.3 which is incompatible.
sktime 0.9.0 requires statsmodels<=0.12.1, but you have statsmodels 0.13.1 which is incompatible.
fracdiff 0.7.0 requires numpy<2.0.0,>=1.21.3, but you have numpy 1.20.3 which is incompatible.
flaml 0.9.0 requires xgboost<=1.3.3,>=0.90, but you have xgboost 1.5.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.

     |████████████████████████████████| 157.5 MB 50 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.5.1
    Uninstalling xgboost-1.5.1:
      Successfully uninstalled xgboost-1.5.1
  Using cached statsmodels-0.12.1-cp37-cp37m-manylinux1_x86_64.whl (9.5 MB)
  Using cached numpy-1.19.3-cp37-cp37m-manylinux2010_x86_64.whl (14.9 MB)


In [48]:
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt

from sklearn import manifold
from sklearn.pipeline import Pipeline

import quandl
import json

import tqdm
from tqdm import tqdm_notebook

from sklearn.ensemble import IsolationForest
import pickle

from sklearn.decomposition import TruncatedSVD

import torch
import torch.nn as nn

import yfinance as yf
yf.pdr_override()

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

import joblib
from joblib import Parallel,delayed

import datetime
from fracdiff.sklearn import FracdiffStat

import sktime

from sktime.forecasting.model_selection import temporal_train_test_split as ttts

import stumpy
#from sklearn.feature_selection import VarianceThreshold

from google.colab import drive
drive.mount('drive',force_remount=True)

from statsmodels.tsa.stattools import adfuller

import optuna
from optuna.samplers import TPESampler

from flaml import AutoML

from tpot import TPOTClassifier

import featexp
from featexp import get_trend_stats

import lightgbm

from lightgbm import LGBMClassifier

import sklearn
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import RFE

from sklearn import model_selection

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

import pytrends
from pytrends import dailydata

import warnings
warnings.simplefilter("ignore")

Mounted at drive


In [3]:
###helper functions

def colon_print(word1,word2):
  print(word1,':',word2)

from sklearn.model_selection import StratifiedKFold, train_test_split

class StratifiedKFold3(StratifiedKFold):

  def split(self, X, y, groups=None):
    s = super().split(X, y, groups)
    for train_indxs, test_indxs in s:
      y_train = y[train_indxs]
      train_indxs, cv_indxs = train_test_split(train_indxs,stratify=y_train, test_size=(1 / (self.n_splits - 1)))
      yield train_indxs, cv_indxs, test_indxs

def split_data(df,targets=None,test_size=0.1):

  df=df.copy()
  
  if targets is not None:
    df['targets']=targets

  x=df.drop('targets',axis=1)
  
  y=df['targets']
  
  xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=test_size, random_state=69, stratify=y)

  train_val_data= pd.concat([xtrain,ytrain],axis=1)

  test_data=pd.concat([xtest,ytest],axis=1)

  return train_val_data,test_data

def ts_split_data(df,targets=None,test_size=0.1):
  df=df.copy()

  if targets is not None:
    df['targets']=targets

  x=df.drop('targets',axis=1)
  
  y=df['targets']

  ytrain, ytest, xtrain, xtest= ttts(y,x,test_size=test_size)

  train_val_data= pd.concat([xtrain,ytrain],axis=1)

  test_data=pd.concat([xtest,ytest],axis=1)

  return train_val_data,test_data



###Data Collection

In [4]:
def update_gtrends():
  gtrends_df=pd.read_csv('/content/drive/MyDrive/Nick Fury/External Datasets/gtrends.csv/google_trends.csv')

  gtrends_df.index=gtrends_df.date

  gtrends_df.index=pd.to_datetime(gtrends_df.index)

  gtrends_df=gtrends_df.drop('date',axis=1)

  next_date=(gtrends_df.index[-1]+datetime.timedelta(1))

  current_date=datetime.datetime.now()

  current_month=current_date.month

  current_year=current_date.year

  month=next_date.month
  year=next_date.year

  gseries_lst=[]
  for term in gtrends_df:
    gtrend_series=pytrends.dailydata.get_daily_data(term,start_year=year,start_mon=month,stop_year=current_year,stop_mon=current_month)[term+'_unscaled']
    gseries_lst.append(gtrend_series)
  
  new_df=pd.concat(gseries_lst,axis=1)
  new_df.columns=gtrends_df.columns

  gtrends_df=pd.concat([gtrends_df,new_df])

  gtrends_df=gtrends_df[~gtrends_df.index.duplicated(keep='first')]

  gtrends_df.to_csv('/content/drive/MyDrive/Nick Fury/External Datasets/gtrends.csv/google_trends.csv')

  return gtrends_df

In [5]:
def get_close_volume(ticker_list):
  returns_lst=[]
  volume_lst=[]
  for ticker in ticker_list:
    data=yf.Ticker(ticker).history(period='max').dropna()
    data=data.drop(['Dividends','Stock Splits'],axis=1)

    data.columns=[ticker+'_'+col for col in data.columns]

    returns_lst.append(data[ticker+'_'+'Close'])
    volume_lst.append(data[ticker+'_'+'Volume'])
  
  close_df=pd.concat(returns_lst,axis=1)
  volume_df=pd.concat(volume_lst,axis=1)
    
  return close_df,volume_df

def get_returns(ticker_list):
  returns_df=pd.DataFrame()
  for ticker in ticker_list:
    data=yf.Ticker(ticker).history(period='max').dropna()
    returns_df[ticker]=data['Close'].pct_change()
  
  return returns_df

def get_data(timeframe='1d',bidirectional=False):

  gtrends_df=pd.read_csv('/content/drive/MyDrive/Nick Fury/External Datasets/gtrends.csv/google_trends.csv')

  gtrends_df.index=gtrends_df.date

  gtrends_df.index=pd.to_datetime(gtrends_df.index)

  gtrends_df=gtrends_df.drop('date',axis=1)

  #index_list=['vox','vcr','vdc','vde','vfh','vht','vis','vgt','vaw','vnq','vpu','dbp','vglt','vgit','vgsh','eem','uso','pdbc','gld','vwo','lqd']

  #returns_df=get_returns(index_list)

  vix_series=pdr.get_data_yahoo('^vix',interval=timeframe)['Close']

  vix_df=pd.DataFrame(vix_series)

  vix_df.columns=['vix_close']
                      
  vix_df['labels']=0
  vix_std=abs(vix_df['vix_close'].diff()).std()

  if not bidirectional:

    #vix_df['labels'][abs(vix_df['vix_close'].diff())>vix_std]=1  #abs
    vix_df['labels'][(vix_df['vix_close'].diff())>vix_std]=1

  else:

    vix_df['labels'][(vix_df['vix_close'].diff())>vix_std]=1  
    vix_df['labels'][vix_df['vix_close'].diff()<-vix_std]=-1

  vix_df['targets']=vix_df['labels'].shift(-1)
  
  quandl.ApiConfig.api_key = '*****************'

  #yale_list=['YALE/US_CONF_INDEX_VAL_INDIV','YALE/US_CONF_INDEX_VAL_INST','YALE/US_CONF_INDEX_CRASH_INDIV','YALE/US_CONF_INDEX_CRASH_INST']

  #yale_df_list=[quandl.get(code, start_date='2010-1-1', end_date=str(datetime.datetime.today().date()))['Index Value'] for code in yale_list] 

  #yale_df=pd.DataFrame(yale_df_list,index=yale_list).T  #yale confidence data

  employ_df=quandl.get('FRED/NROUST', start_date='2010-1-1', end_date=str(datetime.datetime.today().date())) #short term unemployment]
  product_df=quandl.get('FRED/GDPPOT', start_date='2010-1-1', end_date=str(datetime.datetime.today().date())) #nominal gross domestic product

  fed_df=pd.concat([product_df,employ_df],axis=1)
  fed_df.columns=['FRED/NROUST','FRED/GDPPOT'] #federal reserve data

  infl_df=quandl.get('RATEINF/CPI_USA', start_date='2010-1-1', end_date=str(datetime.datetime.today().date()))
  infl_df.columns=['basket_price']
  infl_df['inflation_Q']=infl_df['basket_price'].pct_change()

  short_df=quandl.get('FINRA/FNSQ_'+'spy'.upper(), start_date='2010-1-1', end_date=str(datetime.datetime.today().date()))['ShortVolume']
  short_df.columns=['short_int']

  sentiment_df=quandl.get('AAII/AAII_SENTIMENT-AAII-Investor-Sentiment-Data')

  sentiment_df=sentiment_df.dropna()
  sentiment_df=sentiment_df[['Bullish','Neutral','Bearish']].iloc[1:,:]

  if timeframe=='1d':
  
    to_concat=[short_df,gtrends_df,vix_df] #returns_df,daily
    to_merge=[sentiment_df,infl_df,fed_df,employ_df,product_df]  #yale_df,weekly

  else:
    to_concat=[short_df,gtrends_df] #returns_df,daily
    to_merge=[sentiment_df,infl_df,fed_df,employ_df,product_df,vix_df]  #yale_df,weekly

  combined_df=pd.concat(to_concat,axis=1)

  for df in to_merge:
    combined_df=pd.merge_asof(combined_df,df,right_index=True,left_index=True)

  combined_df=combined_df.astype(float)

  #combined_df=combined_df.fillna(method='pad').dropna()

  combined_df=returns_only(combined_df)

  targets=combined_df['targets']

  combined_df=combined_df.drop(['labels','targets'],axis=1)

  return combined_df,targets

def returns_only(combined_df):

  index_list=['vox','vcr','vdc','vde','vfh','vht','vis','vgt','vaw','vnq','vpu','dbp','vglt','vgit','vgsh','eem','uso','pdbc','gld','vwo','lqd']
  returns_df=get_returns(index_list)

  combined_df=pd.concat([combined_df,returns_df],axis=1)
  
  return combined_df.dropna()

In [ ]:
gtrends_df=update_gtrends()
combined_df,targets=get_data(bidirectional=False,timeframe='1wk')

###Preprocessing

In [7]:
def frac_diff(x, d=0.7):
    """
    Fractionally difference time series

    :param x: numeric vector or univariate time series
    :param d: number specifying the fractional difference order.
    :return: fractionally differenced series
    """
    if np.isnan(np.sum(x)):
        return None

    n = len(x)
    if n < 2:
        return None

    x = np.subtract(x, np.mean(x))

    # calculate weights
    weights = [0] * n
    weights[0] = -d
    for k in range(2, n):
        weights[k - 1] = weights[k - 2] * (k - 1 - d) / k

    # difference series
    ydiff = list(x)

    for i in range(0, n):
        dat = x[:i]
        w = weights[:i]
        ydiff[i] = x[i] + np.dot(w, dat[::-1])

    return np.array(ydiff)

def one_step_fracdiff(close_df,combined_df):
  cutoff_date=combined_df.index[0]

  cutoff_index=len(close_df.loc[:cutoff_date])

  diff_array=np.zeros((combined_df.shape[0],close_df.shape[-1]))

  for t in range(len(diff_array)):
    diff_array[t,:]=frac_diff(close_df.iloc[:cutoff_index+t+1].dropna().values)[-1,:]

  diff_df=pd.DataFrame(diff_array,columns=close_df.columns,index=combined_df.index)

  return diff_df

def update_one_fracdiff():
  
  index_list=['vox','vcr','vdc','vde','vfh','vht','vis','vgt','vaw','vnq','vpu','dbp','vglt','vgit','vgsh','eem','uso','gld','vwo','lqd','spy']
  close_df,vol_df=get_close_volume(index_list)
  
  close_diffs=load_diffs()

  next_date=(close_diffs.index[-1]+datetime.timedelta(1))

  new_diffs=one_step_fracdiff(close_df,close_df.loc[next_date:])

  close_diffs=pd.concat([close_diffs,new_diffs])

  close_diffs.to_csv('/content/drive/MyDrive/close_diff.csv')

  return close_diffs
  
def fracdiff_split_transform(combined_df,targets=None):

  df=combined_df.copy()

  if targets is not None:
    df['targets']=targets

  train_val_data,test_data=ts_split_data(df,targets)

  train_data,val_data=ts_split_data(train_val_data,test_size=0.2)

  index_list=['vox','vcr','vdc','vde','vfh','vht','vis','vgt','vaw','vnq','vpu','dbp','vglt','vgit','vgsh','eem','uso','gld','vwo','lqd','spy']

  close_df,volume_df=get_close_volume(index_list)

  weekly_df=pdr.get_data_yahoo('spy')

  weekly_df.columns=['weekly_'+ col for col in weekly_df.columns]

  weekly_close=weekly_df['weekly_Close']

  weekly_vol=weekly_df['weekly_Volume']

  assets_fracdiffs=fit_fracdiffs(close_df,val_data)

  weekly_fracdiff=fit_fracdiffs(weekly_close,val_data)

  fracdiff_dict={**assets_fracdiffs,**weekly_fracdiff}

  combined_df=pd.concat([df,close_df,volume_df],axis=1)

  weekly_df=pd.concat([weekly_close,weekly_vol],axis=1)

  combined_df=pd.merge_asof(combined_df,weekly_df,right_index=True,left_index=True).dropna()

  for asset in fracdiff_dict:
    asset_diff=fracdiff_dict[asset].transform(combined_df[asset].values.reshape(-1,1))

    combined_df[asset]=asset_diff
  
  return combined_df.drop('targets',axis=1).dropna()

def fit_fracdiffs(close_df,val_data):

  cutoff_date=val_data.index[0]

  if isinstance(close_df,pd.Series):
    close_df=pd.DataFrame(close_df)

  fracdiff_dict={}

  for asset in close_df.columns:
    asset_df=close_df[asset]

    fracdev=FracdiffStat(pvalue=0.01)

    train_asset_close=fracdev.fit(asset_df.loc[:cutoff_date].dropna().values.reshape(-1,1))

    fracdiff_dict[asset]=fracdev

  return fracdiff_dict

def load_diffs():

  close_diffs=pd.read_csv('/content/drive/MyDrive/close_diff.csv')

  close_diffs.index=pd.to_datetime(close_diffs['Unnamed: 0'])
  close_diffs=close_diffs.drop('Unnamed: 0',axis=1)

  return close_diffs

def one_step_combine(combined_df):
  
  close_diffs=load_diffs()
  
  combined_df=pd.concat([combined_df,close_diffs],axis=1)

  return combined_df.dropna()

####Feature Engineering

In [8]:
from sklearn.preprocessing import MinMaxScaler

def scaled_variance_vars(ts_train_data,threshold=0.03):

  scaler=MinMaxScaler()

  scaled_train=scaler.fit_transform(ts_train_data)

  scaled_df=pd.DataFrame(scaled_train,columns=ts_train_data.columns,index=ts_train_data.index)

  return list(scaled_df.var()[scaled_df.var()>=threshold].index)

In [9]:
def agg_features(df,feature):
 # create a bunch of features using the date column

  # create an aggregate dictionary
  aggs = {}
  # for aggregation by month, we calculate the
  # number of unique month values and also the mean
  # we aggregate by num1 and calculate sum, max, min
  # and mean values of this column
  
  num_cols=[col for col in df.columns if col]
  
  for col in num_cols:
    
    aggs[col] = ['mean']

  # for customer_id, we calculate the total count
  #aggs['feature'] = ['size']
  # again for customer_id, we calculate the total unique

  # we group by customer_id and calculate the aggregates
  agg_df = df.groupby(feature).agg(aggs)
  agg_df = agg_df.reset_index()
  
  return agg_df

def match_ts_targets(ts,targets):
  
  ts_copy=ts.copy()
  ts_copy['targets']=targets

  return ts_copy['targets']

def outlier_features(df,ts_train_data):
  if isinstance(df,pd.Series):
      df=pd.DataFrame(df)

  iso=IsolationForest()

  iso.fit(ts_train_data)

  iso_values=iso.predict(df)
  
  return pd.DataFrame(iso_values,index=df.index,columns=['outliers'])

def add_date_features(df):

  date_df=pd.DataFrame(index=df.index)

  if 'date' not in df.columns:
    date_df['date']=df.index

  date_df.loc[:, 'year'] = date_df['date'].dt.year
  date_df.loc[:, 'weekofyear'] = date_df['date'].dt.weekofyear
  date_df.loc[:, 'month'] = date_df['date'].dt.month
  date_df.loc[:, 'dayofweek'] = date_df['date'].dt.dayofweek
  date_df.loc[:, 'weekend'] = (date_df['date'].dt.weekday >=5).astype(int)

  return date_df.drop('date',axis=1)

def poly_featurizer(feat_df,degree=2):

  pf=PolynomialFeatures(degree=degree)

  poly_values=pf.fit_transform(feat_df)

  feat_df=pd.DataFrame(poly_values,columns=['feat_'+ str(i) for i in range(poly_values.shape[-1])],index=feat_df.index)

  return feat_df

def fracdiff_featurizer(combined_df,col_lst,threshold=0.01):

  if isinstance(combined_df,pd.Series):
    combined_df=pd.DataFrame(combined_df)
  
  fracdev=FracdiffStat(pvalue=threshold)

  diff_values=fracdev.fit_transform(combined_df[col_lst])

  diff_df=pd.DataFrame(diff_values,columns=['diff_'+ col for col in col_lst],index=combined_df.index)

  return diff_df

def matprof_featurizer(df,window_size=3,discords=False):

  if isinstance(df,pd.Series):
    df=pd.DataFrame(df)
  
  if not discords:
    pattern='motifs_'

  else:
    pattern='discords_'
  
  if df.shape[-1]>1:
    matprof_array,_= stumpy.mstump(df,m=window_size,discords=discords,normalize=True)

    return pd.DataFrame(matprof_array.T,columns=[pattern+ col for col in df.columns],index=df.index[window_size-1:])
  
  else:
    prof_array,_=stumpy.stump(df,m=window_size,discords=discords,normalize=True)
    
    return  pd.DataFrame(prof_array.T,columns=[pattern+ col for col in df.columns],index=df.index[window_size-1:])

def one_matprof(df,window_size=3,discords=False):
  if isinstance(df,pd.Series):
    df=pd.DataFrame(df)
  
  if not discords:
    pattern='motifs_'

  else:
    pattern='discords_'
  
  
  lst=[simply_mstump(df.iloc[:window_size+t],window_size,discords) for t in range(len(df))]

  return np.stack(lst)
  
def simply_mstump(df,window_size=3,discords=False):
  prof_array,_=stumpy.mstump(df,m=window_size,discords=discords,normalize=True)
  
  return prof_array.T[-1,:]
    
  
class Time2Vec(nn.Module):
  def __init__(self, activation,hiddem_dim,in_features):
      super().__init__()
      self.in_features=in_features
      if activation == "sin":
          self.l1 = SineActivation(self.in_features, hiddem_dim)
      elif activation == "cos":
          self.l1 = CosineActivation(self.in_features, hiddem_dim)
      
      self.fc1 = nn.Linear(hiddem_dim, 2)
  
  def forward(self, x):
      #x = x.unsqueeze(1)
      x = self.l1(x)
      x = self.fc1(x)
      return x

def t2v(tau, f, out_features, w, b, w0, b0, arg=None):
    if arg:
        v1 = f(torch.matmul(tau, w) + b, arg)
    else:
        
        v1 = f(torch.matmul(tau, w) + b)
    v2 = torch.matmul(tau, w0) + b0
   
    return torch.cat([v1, v2], 1)

class SineActivation(nn.Module):
    def __init__(self, in_features, out_features):
        super(SineActivation, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(1, 1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.b = nn.parameter.Parameter(torch.randn(1, out_features-1))
        self.f = torch.sin

    def forward(self, tau):
        return t2v(tau, self.f, self.out_features, self.w, self.b, self.w0, self.b0)

def t2v_func(data):
  if len(data.shape) >1:
    time2vec=Time2Vec(activation='sin',hiddem_dim=100,in_features=data.shape[-1])
    t2v_data=time2vec(torch.Tensor(data.values))

  else:
    data=np.reshape(data.values,(data.shape[0],1))
    time2vec=Time2Vec(activation='sin',hiddem_dim=100,in_features=data.shape[-1])
    t2v_data=time2vec(torch.Tensor(data))

  return pd.DataFrame(t2v_data.detach().numpy(),columns=['t2v_0','t2v_1'],index=data.index)
     

def ts_2_supervised(data, n_in=1, n_out=1, dropnan=True):

  n_vars = 1 if type(data) is list else data.shape[1]
  df = pd.DataFrame(data)
  cols, names = list(), list()
  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
    names += [col+('(t-%d)' % (i)) for col in data.columns]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
    cols.append(df.shift(-i))
    if i == 0:
      names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
    else:
      names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
  # put it all together
  agg = pd.concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
    agg.dropna(inplace=True)
  return agg

def stationarity_tester(series,threshold=0.01,print_mode=False):
  
  p_value=adfuller(series)[1]

  if print_mode:
    if series.name:
     print(series.name+' is')
    
    if p_value<threshold:
      print('Stationary')
    
    else:
      print('not Stationary')
  
  return p_value

def non_stationary_vars(combined_df):
  col_lst=[]

  for col in combined_df.columns:
    p_value=stationarity_tester(combined_df[col],threshold=0.01)
    if p_value>0.01:
      col_lst.append(col)
  
  return col_lst

In [10]:
#feature engineering

def ts_engineer_splits(combined_df):

  ts_train_val_data,ts_test_data=ts_split_data(combined_df,targets)

  ts_train_data,ts_val_data=ts_split_data(ts_train_val_data,test_size=0.2)

  ts_train_data=ts_train_data.drop('targets',axis=1)

  ts_val_data=ts_val_data.drop('targets',axis=1)

  return ts_train_data,ts_val_data

from sklearn.preprocessing import MinMaxScaler

def scaled_variance_vars(ts_train_data,threshold=0.03):

  scaler=MinMaxScaler()

  scaled_train=scaler.fit_transform(ts_train_data)

  scaled_df=pd.DataFrame(scaled_train,columns=ts_train_data.columns,index=ts_train_data.index)

  return list(scaled_df.var()[scaled_df.var()>=threshold].index)

def ts_single_count(df,col):
  df=df.copy()
  df[col+'_count']=0

  for t in range(len(df)):
    relevant=df[col].iloc[:t+1]
    
    temp = relevant.value_counts()

    temp_item=temp[temp.index==relevant[-1]].item()

    df[col+'_count'].iloc[t]=temp_item
  
  return df[col+'_count']

def engineer_splits(df,targets,ts_train_data):

  if 'targets' in df.columns:
    print('target in df')

  proto=df.dropna().copy()

  close_diffs=load_diffs()

  diff_df=one_step_combine(proto)

  scaled_var_vars= scaled_variance_vars(ts_train_data)

  super_df=ts_2_supervised(proto,n_in=2)

  date_df=add_date_features(proto)

  t2v_df=pd.concat([t2v_func(combined_df.iloc[:t+1]).iloc[-1,:] for t in range(len(combined_df))],axis=1).T
  
  iso_df=outlier_features(proto,ts_train_data)

  to_concat=[date_df,iso_df,t2v_df,diff_df,super_df,proto]

  feat_df=pd.concat(to_concat,axis=1).T.drop_duplicates().T

  targets=match_ts_targets(feat_df,targets)

  df=feat_df.copy().dropna()

  df['targets']=targets.dropna().copy()

  x=df.drop('targets',axis=1).T.drop_duplicates().T

  y=df['targets']

  xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=69, stratify=y)

  train_val_data,test_data=split_data(df,targets)

  train_data,val_data=split_data(train_val_data)

  return train_data.T.drop_duplicates().T,val_data.T.drop_duplicates().T,train_val_data.T.drop_duplicates().T,test_data.T.drop_duplicates().T,df

In [11]:
ts_train_data,ts_val_data=ts_engineer_splits(combined_df)

train_data,val_data,train_val_data,test_data,feat_df= engineer_splits(combined_df,targets,ts_train_data)

In [12]:
#this base model was derived using flaml's automl on the train_val data, and serves as the base for feature selection

model=LGBMClassifier(colsample_bytree=0.7339786814213308,
               learning_rate=0.0637270125392533, max_bin=1023,
               min_child_samples=7, n_estimators=663, num_leaves=18,
               reg_alpha=0.0009765625, reg_lambda=0.023076931736850263,
               verbose=-1)

###Feature Selection

In [13]:
def permutation_importance(model,features_valid,labels_valid):
  base_score=roc_auc_score(labels_valid,model.predict(features_valid))

  list_fimportance=[]

  for col in features_valid.columns:
    save=features_valid[col].copy()
    features_valid[col]=np.random.permutation(features_valid[col])
    col_score=roc_auc_score(labels_valid,model.predict(features_valid))

    features_valid[col]=save
    
    list_fimportance.append([col,base_score-col_score])
  
  return pd.DataFrame(list_fimportance,columns=['feature','importance'])

def adversarial_single_selection(df_train, df_test,model, N_val=50000):
    df_train=df_train.copy()
    df_test=df_test.copy()
    
    df_train['targets'] = 0
    df_test['targets'] = 1
    target = 'targets'

    df_master = pd.concat([df_train, df_test], axis=0)
    adversarial_test = df_master.sample(N_val, replace=False)
    adversarial_train = df_master[~df_master.index.isin(adversarial_test.index)]

    model.fit(adversarial_train.drop('targets',axis=1),adversarial_train.targets)

    probs=model.predict_proba(adversarial_test.drop('targets',axis=1))[:,-1]

    return sklearn.metrics.roc_auc_score(adversarial_test.targets,probs),model

def feat_experience(model,df,threshold=0.75,target_col='targets'):

  df=df.copy()

  if targets is not None:
    df['targets']=targets
  
  x=df.drop('targets',axis=1).T.drop_duplicates().T
  y=df.targets

  base_lst=[]
  new_lst=[]
  kf=StratifiedKFold(n_splits=3,shuffle=True,random_state=69)

  print('cv for stat analysis')
  for fold, (t_,v_) in tqdm_notebook(enumerate(kf.split(X=x,y=y))):
    xtrain=x.iloc[t_,:].values
    
    ytrain=y.iloc[t_].values
    
    xval=x.iloc[v_,:].values
    yval=y.iloc[v_].values
  
    model.fit(xtrain,ytrain)

    probs=model.predict_proba(xval)[:,-1]

    baseline_score=sklearn.metrics.roc_auc_score(yval,probs)

    base_lst.append(baseline_score)

    train_data=pd.concat([pd.DataFrame(xtrain),pd.Series(ytrain)],axis=1).T.drop_duplicates().T

    cols=list(x.columns)
    cols.append(y.name)

    train_data.columns=cols

    test_data=pd.concat([pd.DataFrame(xval),pd.Series(yval)],axis=1)
    test_data.columns=cols

    trend_df=get_trend_stats(data=train_data,data_test=test_data,target_col=target_col)

    good_feats=list(trend_df['Feature'][abs(trend_df['Trend_correlation'])>threshold])
   
    good_xtrain=x[good_feats].iloc[t_,:]
    good_xval=x[good_feats].iloc[v_,:]

    model.fit(good_xtrain.values,ytrain)

    probs=model.predict_proba(good_xval)[:,-1]

    new_score=sklearn.metrics.roc_auc_score(yval,probs)

    new_lst.append(new_score)
  
  mean_new_score=np.mean(new_lst)
  mean_base_score=np.mean(base_lst)
  
  if mean_new_score>mean_base_score:
    return set(df.columns)-set(good_feats)
  
  else:
    print('futile')
    return None

def nested_feature_selection(df,model,targets=None):
  
  df=df.copy()

  if targets is not None:
    df['targets']=targets
  
  features=df.drop('targets',axis=1).columns
  
  folds_df=create_folds(df)

  prime_train_list=[]
  prime_val_list=[]

  print('nested cv for feature selection')
  for fold in tqdm_notebook(folds_df['kfold'].unique()): #outer cv

    df_train_outer = folds_df[folds_df.kfold != fold].reset_index(drop=True)
 
    df_valid_outer = folds_df[folds_df.kfold == fold].reset_index(drop=True)
    
    df_inner= create_folds(df_train_outer)

    val_scores_list=[]
    train_scores_list=[]
    
    print('doing inner folds')
    for inner_fold in tqdm_notebook(df_inner['kfold'].unique()):
      df_train_inner= df_inner[df_inner.kfold != inner_fold].reset_index(drop=True)

      df_valid_inner=df_inner[df_inner.kfold == inner_fold].reset_index(drop=True)
      
      val_series=pd.Series()
      train_series=pd.Series()
      for col in features:
        xtrain_inner=np.expand_dims(df_train_inner[col].values,axis=1)
        ytrain_inner=df_train_inner.targets.values

        xval_inner=np.expand_dims(df_valid_inner[col].values,axis=1)
        yval_inner=df_valid_inner.targets.values
    
        model.fit(xtrain_inner,ytrain_inner)

        val_probs=model.predict_proba(xval_inner)[:,-1]

        train_probs=model.predict_proba(xtrain_inner)[:,-1]

        val_score=sklearn.metrics.roc_auc_score(yval_inner,val_probs)

        train_score=sklearn.metrics.roc_auc_score(ytrain_inner,train_probs)
      
        val_series.loc[col]=val_score
        train_series.loc[col]=train_score

      val_scores_list.append(val_series)
      train_scores_list.append(train_series)
  
    prime_train_list.append(pd.concat(train_scores_list,axis=1))
    
    prime_val_list.append(pd.concat(val_scores_list,axis=1))
  
  new_train_list=[i.T.mean() for i in prime_train_list]
  new_val_list=[i.T.mean() for i in prime_val_list]

  train_feat_df=sum(new_train_list)/len(new_train_list)

  val_feat_df=sum(new_val_list)/len(new_val_list)

  comb_df=pd.concat([train_feat_df,val_feat_df],axis=1)

  bad_list=[]

  for var in comb_df.T:
    if comb_df.T[var].iloc[0]<0.5 or comb_df.T[var].iloc[-1]<0.5:
      bad_list.append(var)
  
  return bad_list

def adversarial_validation(train_val_data,model):

  df=train_val_data.copy()

  df['targets']=-1

  x=df.drop('targets',axis=1).reset_index(drop=True)
  y=df.targets.reset_index(drop=True)

  xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=69, stratify=y)

  ytrain=pd.DataFrame(np.zeros(ytrain.shape),columns=['targets'],index=ytrain.index).astype(int)

  ytest=pd.DataFrame(np.ones(ytest.shape),columns=['targets'],index=ytest.index).astype(int)

  new_train=pd.concat([xtrain,ytrain],axis=1)

  new_test=pd.concat([xtest,ytest],axis=1)

  new_data=pd.concat([new_train,new_test])

  new_x=new_data.drop('targets',axis=1).values

  new_y=new_data.targets.values

  kf=StratifiedKFold(n_splits=3,shuffle=True,random_state=69)

  score_lst=[]

  print('cv for adversarial feature selection')
  for fold, (t_,v_) in tqdm_notebook(enumerate(kf.split(X=new_x,y=new_y))):
    xtrain=new_x[t_,:]
    ytrain=new_y[t_]
    
    xval=new_x[v_,:]
    yval=new_y[v_]

    model.fit(xtrain,ytrain)

    probs=model.predict_proba(xval)[:,-1]

    score=sklearn.metrics.roc_auc_score(yval,probs)

    score_lst.append(score)
  
  if np.mean(score_lst)>0.6:
    xtrain,xtest,ytrain,ytest=train_test_split(new_x,new_y,random_state=69,stratify=new_y)
    final_model=model.fit(xtrain,ytrain)

    return final_model,np.mean(score_lst)

  else:
    return None,np.mean(score_lst)

def time_consistency(ts_df,model,targets=None):

  df=ts_df.copy()

  if targets is not None:
    df['targets']=targets

  train_data=df.iloc[:round(0.7*len(df)),:]

  first_data=train_data.iloc[:30]

  last_data=train_data.iloc[-30:]

  bad_lst=[]
  
  print('Time consistency analysis for each feature')
  for col in tqdm_notebook(train_data.columns):
    
    first_x=np.expand_dims(first_data[col].values,axis=1)
    first_y=np.expand_dims(first_data.targets.values,axis=1)

    last_x=np.expand_dims(last_data[col].values,axis=1)
    last_y=np.expand_dims(last_data.targets.values,axis=1)

    model.fit(first_x,first_y)

    first_probs=model.predict_proba(first_y)[:,-1]

    last_probs=model.predict_proba(last_x)[:,-1]

    first_score=sklearn.metrics.roc_auc_score(first_y,first_probs)

    last_score=sklearn.metrics.roc_auc_score(last_y,last_probs)

    if first_score <0.5 or last_score<0.5:
      bad_lst.append(col)

  return bad_lst                

In [ ]:
inconsistent_lst=time_consistency(train_data,model)

bad_trend_feats=feat_experience(model,train_data,threshold=0.5)

adv_model,adv_score=adversarial_validation(train_val_data,model)

nfs_lst=nested_feature_selection(train_val_data,model)

In [ ]:
feature_combos=[bad_trend_feats,inconsistent_lst+list(bad_trend_feats),inconsistent_lst,inconsistent_lst+nfs_lst+list(bad_trend_feats),list(bad_trend_feats)+nfs_lst,
nfs_lst,inconsistent_lst+nfs_lst]

score_lst=[]
f_lst2=[]
for combo in feature_combos:
  fold_lst=run_folds(train_val_data.drop(combo,axis=1),model=model,targets=targets,pref_metric='precision')

  prec_lst=[]
  f_lst=[]
  for fold in fold_lst:
    prec_lst.append(fold_lst[fold]['scores']['average_precision'])
    f_lst.append(fold_lst[fold]['scores']['overall_f0.5'])

  score_lst.append(np.mean(prec_lst))
  f_lst2.append(np.mean(f_lst))


#just take the argmax(index) of either lists to see which combination of features to drop maximizes the validation score of the preferred metric

In [16]:
def load_weekly_to_drop(to_save=None):

  if to_save is not None:
    with open('/content/drive/MyDrive/drop_columns.txt', 'w') as fp:
      json.dump(to_save,fp)

  with open('/content/drive/MyDrive/drop_columns.txt', 'r') as fp:
    to_drop=json.load(fp)

  return to_drop

In [ ]:
to_drop=load_weekly_to_drop()

###Automl

In [17]:
def run_nested_flaml(df,targets=None,seconds=300):
  
  df=df.copy()

  if targets is not None:
    df['targets']=targets
  
  folds_df=create_folds(df)

  model_lst=[]

  for fold in tqdm_notebook(folds_df['kfold'].unique()): #outer cv

    df_train_outer = folds_df[folds_df.kfold != fold].reset_index(drop=True)

    df_valid_outer = folds_df[folds_df.kfold == fold].reset_index(drop=True)

    xtrain=df_train_outer.drop(['targets','kfold'],axis=1).values
    ytrain=df_train_outer.targets.values
    
    automl=AutoML()

    automl_settings = {
        "time_budget": seconds,  # in seconds
        "metric": 'ap',
        "task": 'classification',
    }

    automl.fit(xtrain,ytrain,**automl_settings)

    model_lst.append(automl.model.estimator)
    
  return cv_eval_models(df,model_lst,target_metric)

def run_nested_tpot(df,targets=None):
  
  df=df.copy()

  if targets is not None:
    df['targets']=targets
  
  folds_df=create_folds(df)

  model_lst=[]

  for fold in tqdm_notebook(folds_df['kfold'].unique()): #outer cv

    df_train_outer = folds_df[folds_df.kfold != fold].reset_index(drop=True)

    df_valid_outer = folds_df[folds_df.kfold == fold].reset_index(drop=True)

    xtrain=df_train_outer.drop(['targets','kfold'],axis=1).values
    ytrain=df_train_outer.targets.values
    
    automl=AutoML()

    cv = StratifiedKFold(n_splits=5, random_state=69,shuffle=True)

    tpot = TPOTClassifier(generations=5, population_size=10, verbosity=2, random_state=69,cv=cv,scoring='average_precision')

    tpot.fit(xtrain, ytrain)

    model_lst.append(tpot.fitted_pipeline_)

  return model_lst
  #return cv_eval_models(model_lst,target_metric=target_metric)
        
global target_metric
target_metric='average_precision'

def cv_eval_models(df,model_lst,target_metric):
  
  model_scores=[]
  for model in model_lst:
    try:
      fold_lst=run_folds(df,model,pref_metric='precision')
      
      fold_scores=[]
      for fold in fold_lst:
        fold_scores.append(fold_lst[fold]['scores'][target_metric])    

      model_scores.append(np.mean(fold_scores))
    
    except Exception as e:
      print(e)
      pass

  return model_lst[np.argmax(model_scores)]

In [ ]:
best_model=LGBMClassifier(colsample_bytree=0.3794507019586446,
               learning_rate=0.019289262657833043, max_bin=255,
               min_child_samples=2, n_estimators=132, num_leaves=389,
               reg_alpha=0.0009765625, reg_lambda=0.011000563062299913,
               verbose=-1)

In [ ]:
best_model=run_nested_flaml(train_val_data.drop(to_drop,axis=1),targets=targets,seconds=300)
tpot_model=run_nested_tpot(train_val_data.drop(to_drop,axis=1),targets=targets)

In [ ]:
fold_lst=run_folds(train_val_data.drop(to_drop,axis=1),model=best_model,targets=targets,pref_metric='precision')

In [ ]:
best_model.fit(train_val_data.drop(to_drop,axis=1),train_val_data.targets)
probs=best_model.predict_proba(test_data.drop(to_drop,axis=1))[:,-1]

print(sklearn.metrics.average_precision_score(test_data.targets,probs))

In [ ]:
def matters(model,test_data):
  probs=model.predict_proba(test_data.drop('targets',axis=1))[:,-1]
  
  preds=model.predict(test_data.drop('targets',axis=1))
  
  return sklearn.metrics.average_precision_score(test_data['targets'],probs),sklearn.metrics.fbeta_score(test_data['targets'],preds,beta=0.5)

In [45]:
def predict_next_week(feat_df):

  best_threshold=0.5

  best_model=pickle_jar()

  to_drop=load_weekly_to_drop()

  cols_to_use=train_val_data.drop(to_drop,axis=1).columns

  dropped_df=feat_df[cols_to_use]

  best_model.fit(dropped_df.drop('targets',axis=1).iloc[:-1],feat_df.targets.iloc[:-1])

  prob=best_model.predict_proba(dropped_df.drop('targets',axis=1).iloc[-1].values.reshape(1,-1))[:,-1]

  if prob>best_threshold:
    print('Positive increase in volatility next week')

  elif prob<1-best_threshold:
    print('No positive increase in volatility next week')

In [46]:
predict_next_week(feat_df

No positive increase in volatility next week


In [18]:
def pickler(model):
  with open('/content/drive/MyDrive/weekly_pos_model.pkl','wb') as file:
    pickle.dump(model, file)

def pickle_jar():
  with open('/content/drive/MyDrive/weekly_pos_model.pkl','rb') as file:
     model= pickle.load(file)
  
  return model

####Run Cross Validation


In [36]:
def create_folds(df,targets=None,n_folds=5):

  df=df.copy()
  
  if targets is not None:
    df['targets']=targets
  
  
  df["kfold"] = -1
 
  df = df.sample(frac=1).reset_index(drop=True)
 
  y = df.targets.values
 
  kf=StratifiedKFold(n_splits=n_folds,random_state=69,shuffle=True)

  for f, (tr_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'kfold'] = f
    
  return df

def run_fold(df,model,fold):
  df_train = df[df.kfold != fold].reset_index(drop=True)

  df_valid = df[df.kfold == fold].reset_index(drop=True)

  xtrain=df_train.drop(['targets','kfold'],axis=1).values
  ytrain=df_train.targets.values

  
  xval=df_valid.drop(['targets','kfold'],axis=1).values
  yval=df_valid.targets.values

  model.fit(xtrain,ytrain)

  return model,(xval,yval)

def run_folds(df,model,metric=None,targets=None,n_folds=5,task='classification',pref_metric=None,param_grid=None):

  print('model:',model)
  print('task:',task.upper())

  df=df.copy()

  if targets is not None:
    df['targets']=targets
  
  fold_df=create_folds(df,n_folds=n_folds)
  
  fold_dict={}

  if task=='classification':
    if pref_metric is None:
      print('Which one is most important? recall,precision,or neither?')
      pref_metric=input()

    for fold in range(n_folds):
      mini_dict={}
      scores_dict={}
      
      print('--'*10)
      
      model,(xval,yval)=run_fold(fold_df,model,fold)
      labels=model.classes_

      if param_grid is not None:
        model,best_params,(xval,yval)=run_nested_fold(fold_df,model,fold,param_grid)
      
      if len(df['targets'].unique())>2:
        
        probs= model.predict_proba(xval)
        
        scores_dict['multi_brier_loss']=multi_brier_loss(yval,probs,labels)
        scores_dict['average_precision']=avg_precision(thresh_df)
     
      elif len(df['targets'].unique())<=2:

        probs = model.predict_proba(xval)[:,-1]
        
        scores_dict['brier_loss']=sklearn.metrics.brier_score_loss(yval,probs)
        scores_dict['average_precision']=sklearn.metrics.average_precision_score(yval,probs,average='macro')

      thresh_df=threshold_matrix(yval,probs,labels)

      overall_dict=overall_class_metrics(thresh_df)

      pref_dict=pref_metrics(pref_metric,thresh_df)
      
      scores_dict['roc_auc']=sklearn.metrics.roc_auc_score(yval,probs,average='macro',multi_class='ovr')

      scores_dict={**scores_dict,**pref_dict,**overall_dict}
    
      for score in scores_dict:
        colon_print(score,scores_dict[score])
   
  #elif task=='regression':
    ###
    
      if metric is not None:
        print(str(metric)+':',metric(yval,probs))
      
      mini_dict['val']=(xval,yval,probs)
      mini_dict['scores']=scores_dict
      mini_dict['model']=model

      if param_grid is not None:
        mini_dict['best_params']=best_params
        
      fold_dict[fold]=mini_dict

  return fold_dict

####Metrics

In [37]:
def multi_brier_loss(targets, probs,labels):
  pad_lst=[]

  for target in targets:
    pad_lst.append((labels==target).astype('int'))
  
  pad_targets=np.array(pad_lst)

  return np.mean(np.sum((probs - pad_targets)**2,axis=1))

def thresholder(probs,threshold,labels):
  
  filter=(probs>=threshold).astype('int')

  if len(filter.shape)>1:
    return labels[np.argmax(filter,axis=1)]
  
  else:
    return filter

def best_thresholds(thresh_df):

  recall_thresh=thresh_df.index[thresh_df['recall'].argmax()]
  precision_thresh=thresh_df.index[thresh_df['precision'].argmax()]
  f1_thresh=thresh_df.index[thresh_df['f1'].argmax()]
  fhalf_thresh=thresh_df.index[thresh_df['f0.5'].argmax()]
  f2_thresh=thresh_df.index[thresh_df['f2'].argmax()]

  return f1_thresh,fhalf_thresh,f2_thresh,recall_thresh,precision_thresh  

def best_class_scores(thresh_df):
  best_recall=thresh_df['recall'].max()
  best_precision=thresh_df['precision'].max()
  best_f1=thresh_df['f1'].max()
  best_f2=thresh_df['f2'].max()
  best_fhalf=thresh_df['f0.5'].max()

  return best_recall,best_precision,best_f1,best_f2,best_fhalf

def threshold_matrix(ytest,probs,labels):
  recall_lst=[] 
  precision_lst=[]
  mcc_lst=[]
  kappa_lst=[]

  for threshold in np.arange(0.5,1,0.05):

    preds=thresholder(probs,threshold,labels)

    cm=sklearn.metrics.confusion_matrix(ytest,preds,labels=labels) 
   
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    tn= cm.sum()-(tp+fp+fn)
    
    if len(labels)==2:
      tp=max(tp)
      fp=max(fp)
      fn=max(fn)
      tn=max(tn)
    
    precision=np.mean(tp/(tp+fp))
  
    recall=np.mean(tp/(tp+fn))
    
    mcc=sklearn.metrics.matthews_corrcoef(ytest,preds)

    try:
      kappa=sklearn.metrics.cohen_kappa_score(ytest,preds)
    except:
      kappa=np.nan
      pass
        
    recall_lst.append(recall)
    precision_lst.append(precision)
    mcc_lst.append(mcc)
    kappa_lst.append(kappa)

  thresh_df=pd.DataFrame(precision_lst,columns=['precision'],index=list(np.arange(0.5,1,0.05)))
  thresh_df['recall']=recall_lst
  thresh_df['mcc']=mcc_lst
  thresh_df['kappa']=kappa_lst

  thresh_df['f1']=(2*thresh_df['recall']*thresh_df['precision'])/(thresh_df['recall']+thresh_df['precision'])

  thresh_df['f0.5']=((1+0.5**2)*thresh_df['recall']*thresh_df['precision'])/((0.5**2)*thresh_df['precision']+thresh_df['recall'])

  thresh_df['f2']=((1+2**2)*thresh_df['recall']*thresh_df['precision'])/((2**2)*thresh_df['precision']+thresh_df['recall'])

  return thresh_df

def avg_precision(thresh_df):
  
  try:
    avg_precision=sklearn.metrics.auc(thresh_df['recall'],thresh_df['precision'])
      
  except:
  
    avg_precision=np.nan
    pass
  
  return avg_precision

def overall_class_metrics(thresh_df):
  scores_dict={}
  scores_dict['f1_score']=thresh_df['f1'].mean()
  
  scores_dict['mcc']=thresh_df['mcc'].mean()
  scores_dict['cohen_kappa']= thresh_df['kappa'].mean()

  return scores_dict

def pref_metrics(pref_metric,thresh_df):
  f1_thresh,fhalf_thresh,f2_thresh,recall_thresh,precision_thresh=best_thresholds(thresh_df)
  best_recall,best_precision,best_f1,best_f2,best_fhalf=best_class_scores(thresh_df)

  scores_dict={}
  
  if pref_metric.lower()=='recall':
    
    scores_dict['recall_thresh']=recall_thresh
    scores_dict['best_recall']=best_recall

    scores_dict['f2_thresh']= f2_thresh
    scores_dict['best_f2']= best_f2
    
    scores_dict['overall_f2']=thresh_df['f2'].mean()

  elif pref_metric.lower()=='precision':
    
    scores_dict['precision_thresh']=precision_thresh
    scores_dict['best_precision']=best_precision
  
    scores_dict['f0.5_threshold'] =fhalf_thresh
    scores_dict['best_f0.5']=best_fhalf
    
    scores_dict['overall_f0.5']=thresh_df['f0.5'].mean()
    
  else:
    scores_dict['f1_threshold']=f1_thresh
    scores_dict['best_f1']= best_f1

  return scores_dict